In [15]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import KBinsDiscretizer

from torch.nn import functional as F
import bisect

torch.manual_seed(1337)

# hyperparameters
batch_size = 16 
block_size = 20 
max_iters = 5000
epochs = 5
eval_interval = 100
learning_rate = 1e-3


st_dims = 4
ac_dims = 1
rw_dims = 1
rtg_dims= 1

traj_len = 7

rw_cho = 1+1
ac_cho = 2
rtg_cho = 150+1

eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
data = pd.read_csv("cartpole_play.csv")

In [17]:
rename = {'0':'episode', '1':'obs1', '2':'obs2', '3':'obs3', '4':'obs4', '5':'actions', '6':'rewards', '7':'rewardstg'}
data = data.rename(rename, axis=1)
data.drop(columns='Unnamed: 0', inplace=True)
data

,episode,obs1,obs2,obs3,obs4,actions,rewards,rewardstg
0,0.0,0.048681,0.233331,-0.044224,-0.267353,1.0,1.0,66.0
1,0.0,0.053347,0.038867,-0.049571,0.011060,0.0,1.0,65.0
2,0.0,0.054124,-0.155511,-0.049350,0.287700,0.0,1.0,64.0
3,0.0,0.051014,-0.349895,-0.043596,0.564419,0.0,1.0,63.0
4,0.0,0.044016,-0.154190,-0.032307,0.258327,1.0,1.0,62.0
...,...,...,...,...,...,...,...,...
36762,1099.0,0.145006,0.416619,-0.229412,-1.206810,1.0,1.0,4.0
36763,1099.0,0.145006,0.416619,-0.229412,-1.206810,1.0,1.0,3.0
36764,1099.0,0.145006,0.416619,-0.229412,-1.206810,0.0,1.0,2.0
36765,1099.0,0.145006,0.416619,-0.229412,-1.206810,0.0,1.0,1.0


In [18]:
observations = data[['obs1', 'obs2', 'obs3', 'obs4']]
actions = data['actions']
rewards = data['rewards']
rewardstg = data['rewardstg']

In [19]:
#this class will work with numpy arrays as inputs, except fit_transform
class DataDiscretizer:
    def __init__(self, vocab, traj_len, encode='ordinal', strategy='uniform'):
        self.vocab = vocab
        self.traj_len = traj_len
        self.ob_enc = KBinsDiscretizer(vocab, encode="ordinal", strategy='uniform')
        
    def fit_transform(self, observations, actions, rewards, rewardstg):
        self.dif = np.array([i*vocab for i in range(st_dims)])
        sim_ob_tok = pd.DataFrame(self.ob_enc.fit_transform(observations.values))
        sim_tok = pd.concat([sim_ob_tok, actions, rewards, rewardstg], axis=1)
        dif_ob_tok = sim_ob_tok.copy()
        for i, c in enumerate(dif_ob_tok):
            dif_ob_tok[c] = sim_ob_tok[c] + self.dif[i]
        dif_tok = pd.concat([dif_ob_tok, actions, rewards, rewardstg], axis=1)
        self.total_vocab = dif_tok.max().max()
        return dif_tok.values.reshape(-1).astype(int), sim_tok.values.reshape(-1).astype(int)
        
    def discretize_observation(self, obs):
        return self.ob_enc.transform(obs.reshape(1, -1)).reshape(-1)
        
    def discretize_trajectory(self, traj):
        obs = traj[:st_dims]
        rest = traj[st_dims:]
        print(obs, rest)
        sim_obs_tok = self.discretize_observation(obs)
        dif_obs_tok = sim_obs_tok + self.dif
        sim_tok = np.concatenate([sim_obs_tok, rest])
        dif_tok = np.concatenate([dif_obs_tok, rest])
        return sim_tok.astype(int), dif_tok.astype(int)
    
    def similar_to_different(self, tok, mod):
        mod = mod%self.traj_len
        if mod < st_dims:
            return tok
        else:
            return tok+self.dif[mod]
        
    def get_vocab_size(self):
        return self.total_vocab
        

In [20]:
dd = DataDiscretizer(vocab ,traj_len)
dd.fit_transform(observations, actions, rewards, rewardstg)

(array([ 57, 153, 242, ...,   1,   1,   0]),
 array([57, 53, 42, ...,  1,  1,  0]))

In [7]:
st_cho = int(dd.get_vocab_size())

In [8]:
def get_batch():
    ix = torch.randint(len(discretized) - block_size, (batch_size,))
    x = torch.stack([dif[i:i+block_size] for i in ix])
    y = torch.stack([sim[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y, ix%traj_len

@torch.no_grad()
def estimate_loss():
    out = {}
    tt.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        x, y = get_batch()
        logits, loss = tt(x, targets=y)
        losses[k] = loss.item()
    out = losses.mean()
    tt.train()
    return out

In [9]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(3*block_size, 3*block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [10]:
# super simple bigram model
class TrajectoryTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.st_emb = nn.Embedding(st_cho, n_embd)
        self.ac_emb = nn.Embedding(ac_cho, n_embd)
        self.rw_emb = nn.Embedding(rw_cho, n_embd)
        self.rtg_emb= nn.Embedding(rtg_cho, n_embd)
        
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        
        self.st_head = nn.Linear(n_embd, st_cho)
        self.ac_head = nn.Linear(n_embd, ac_cho)
        self.rw_head = nn.Linear(n_embd, rw_cho)
        self.rtg_head = nn.Linear(n_embd,rtg_cho)

    def forward(self, sequence, targets=None):
        vocab_embedding = self.embedding(sequence)#(B,T,C)
        B, T, C = vocab_embedding.shape
        pos_encoding = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = vocab_embedding + pos_encoding #(B,T,C)
        x = self.blocks(x) # (B,T,C)
        states, actions, rewards, rewardstg = self.vocabbing(x) 

        preds = self.lm_head(x) # (B,T,vocab)

        if targets is None:
            loss = None
        else:
            B, T, C = preds.shape
            preds = preds.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(preds, targets)

        return preds, loss
    
    def generate_masks(self, mods):
        st_mask = torch.stack([(torch.arange(block_size) + mod) % 7 < 4 for mod in mods])
        ac_mask = torch.stack([(torch.arange(block_size) + mod) % 7 == 4 for mod in mods])
        rw_mask = torch.stack([(torch.arange(block_size) + mod) % 7 == 5 for mod in mods])
        rtg_mask = torch.stack([(torch.arange(block_size) + mod)% 7 == 6 for mod in mods])
        return st_mask, ac_mask, rw_mask, rtg_mask

        
    def embedding(self, sequences, mods):
        st_mask, ac_mask, rw_mask, rtg_mask = self.generate_masks(mods)
        inp = torch.ones([batch_size, block_size, n_embd])
        inp[st_mask] = self.st_emb(sequences[st_mask])
        inp[ac_mask] = self.ac_emb(sequences[ac_mask])
        inp[rw_mask] = self.rw_emb(sequences[rw_mask])
        inp[rtg_mask] = self.rtg_emb(sequences[rtg_mask])
        ret = inp.reshape([batch_size, block_size, n_embd])
        return ret
    
    def vocabbing(self, out, mods):
        st_mask, ac_mask, rw_mask, rtg_mask = self.generate_masks(mods+1)
        states = self.st_head(out[st_mask]).reshape(batch_size, -1, st_cho)
        actions = self.ac_head(out[ac_mask]).reshape(batch_size, -1, ac_cho)
        rewards = self.rw_head(out[rw_mask]).reshape(batch_size, -1, rw_cho)
        rewardstg = self.rtg_head(out[rtg_mask]).reshape(batch_size, -1, rtg_cho)
        return states, actions, rewards, rewardstg
        
    def loss(self, states, actions, rewards, rewardstg, targets):
        loss = 0
        st_mask, ac_mask, rw_mask, rtg_mask = self.generate_masks(mods)
        loss += F.cross_entropy(targets[st_mask], states)
        loss += F.cross_entropy(targets[ac_mask], actions)
        loss += F.cross_entropy(targets[rw_mask], rewards)
        loss += F.cross_entropy(targets[rtg_mask], rewardstg)
        return loss

        
    


In [11]:
st_cho

399

In [12]:
tt = TrajectoryTransformer()
tt.to(device)
optimizer = torch.optim.AdamW(tt.parameters(), lr=learning_rate)

In [13]:
11904/(185)

64.34594594594594

In [14]:
x, y, mod = get_batch()
evaluate = tt.embedding(x, mod)

NameError: name 'discretized' is not defined

In [ ]:
for _ in range(epochs):
    for iteration in range(5000):
      # every once in a while evaluate the loss on train and val sets
        if iteration % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iteration}: train loss {losses:.4f}")

        # sample a batch of data
        x, y = get_batch()

        # evaluate the loss
        logits, loss = tt(x, targets=y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

In [ ]:
save_path = 'tt_instance_1.model'

# Save the model's state dictionary to the file
torch.save(tt.state_dict(), save_path)

In [ ]:
state_dict = torch.load(save_path)
tt.load_state_dict(state_dict)

In [ ]:
trajectory_length = st_dims + ac_dims + 2*rw_dims

In [ ]:
context = discretized.view(-1, trajectory_length)[35]

In [ ]:
prevs = prevs.astype(int)
prevs

In [ ]:
def token_to_discretized(tk, i):
    return tk + prevs[i]

def trajectory_to_token(traj):
    return traj-prevs

In [ ]:
import torch
import numpy as np

def beam_search(model, context, st_width=3, ac_width=4):
    if type(context) != list and type(context) != np.ndarray:
        context = context.detach().cpu().numpy()
    window_by_index = [st_width] * st_dims + [ac_width] * ac_dims + [1] + [1]
    top_beams_context = [context]
    top_beams_prob = [0]

    for i in range(trajectory_length):
        square_beams_context = []
        square_beams_prob = []
        k = window_by_index[i]

        for seq, prob in zip(top_beams_context, top_beams_prob):
            logits, _ = model(torch.tensor(seq[-block_size:], device=device).unsqueeze(0))
            logits = F.softmax(logits[0, -1, :], dim=0).detach().cpu().numpy()

            top_k = np.argpartition(logits, -k)[-k:]
            square_beams_prob.extend(logits[top_k] + prob)

            top_dis_k = [token_to_discretized(tk, i) for tk in top_k]
            square_beams_context.extend([np.concatenate([seq, tk.reshape(1)]) \
                                  for tk in top_dis_k])

        if i < st_dims:
            idxs = np.argsort(square_beams_prob)[-k:]
        else:
            idxs = np.argsort(square_beams_prob)

        top_beams_context = np.array(square_beams_context)[idxs]
        top_beams_prob = np.array(square_beams_prob)[idxs]
    
    best_traj = None
    traj_rwds = []
    
    for c in top_beams_context:
        traj = c[-trajectory_length:]
        traj = trajectory_to_token(traj)
        traj_rwd = kbd_rewardstg.inverse_transform((traj[-1]).reshape(-1,1)).reshape(-1)[0]
        traj_rwds.append(traj_rwd)
    
    rwds_idx = np.argsort(traj_rwds)
    top_beams_context = top_beams_context[rwds_idx]
    top_beams_prob = top_beams_prob[rwds_idx]

    return top_beams_context, top_beams_prob

def extract_action(trajectory):
    return trajectory[st_dims::trajectory_length]


In [ ]:
discretized.reshape(-1,trajectory_length)[:,5]

In [ ]:
# print(context)
top_beams_context,  top_beams_prob = beam_search(tt, context, 3, 4)
print(top_beams_context.shape, top_beams_prob.shape)
# temp = top_beams_context[-1]
# print(temp)
# print(sorted(rwds))

# Test

In [ ]:
import gymnasium as gym

In [ ]:
def to_tokens(traj):
    observations = kbd_observations.transform(np.array(traj[0:4]).reshape(1, -1)).reshape(-1)
    action = np.array(traj[4]).reshape(-1)
    reward = np.array(traj[5]).reshape(-1)
    rewardtg = kbd_rewardstg.transform(np.array(traj[6]).reshape(1, -1)).reshape(-1)
    tokenized = np.concatenate([observations, action, reward, rewardtg])
    tokenized += prevs 
    return tokenized.astype(int)

In [ ]:
example = np.array([0.001888, 0.033522, -0.041096, -0.040241, 1.0, 1.0, 21.0])
to_tokens(example)

In [ ]:
env = gym.make("CartPole-v1")


def testing(model, context):
    total_reward = 0.0
    desired_reward = 200

    _, _ = env.reset(seed=None)
    action = 0
    state, reward, terminated, _, _ = env.step(action)
    terminated = False
    time = 0
    
    state_ = state.reshape(-1)
    action_ = np.array(action).reshape(-1)
    reward_ = np.array(reward).reshape(-1)
    desired_reward_ = np.array(desired_reward).reshape(-1)

    to_encode = np.concatenate([state_, action_, reward_, desired_reward_])
    context = to_tokens(to_encode) 
    print(context)
    
    while not terminated and time <= 500:
        seqs, probs = beam_search(tt, context)
#         print(seqs)
        action = -2
        seq = 'a'
        for f in range(11, 0, -1):
            seq = seqs[f][-trajectory_length:]
            seq = seq - prevs
            action = int(seq[4])
            if (action == 0 or action == 1):
                break
#         print(seq+prevs)
#         print(action)
        
        state, reward, terminated, _, _ = env.step(action)
        total_reward += reward
        desired_reward -= reward

        state_ = state.reshape(-1)
        action_ = np.array(action).reshape(-1)
        reward_ = np.array(reward).reshape(-1)
        desired_reward_ = np.array(desired_reward).reshape(-1)

        to_encode = np.concatenate([state_, action_, reward_, desired_reward_])
        context = to_tokens(to_encode) 
        
        time += 1
#         if time%1 == 0:
#             print(time)

      # Print reward
    print("total_reward = {}".format(total_reward))
    env.close()

In [ ]:
prevs

In [ ]:
testing(tt, context)

In [ ]:
test = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

test[1:3:5]

In [ ]:
import numpy as np

input_array = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

result = input_array[np.arange(0, len(input_array), 7)].reshape(-1, 4)

In [ ]:
mod = 0
st_mask = [(x+mod)%7 < 4 for x in range(11)]
ac_mask = [(x+mod)%7 == 4 for x in range(33)]
rw_mask = [(x+mod)%7 == 5 for x in range(33)]
rtg_mask= [(x+mod)%7 == 6 for x in range(33)]

test = np.array(range(11))
print(test[st_mask])

In [ ]:
ll = nn.Linear(5, 2)
inp = torch.tensor(np.random.rand(11, 5), dtype=torch.float)
out = ll(inp)
print(out.shape)
print(out)
out[st_mask] = torch.zeros(st_mask.shape)

In [ ]:
mask

In [ ]:
import numpy as np

mods = np.array([0, 1, 2])  # Replace with your desired tensor of mods

length = 14  # Length of the masks

st_mask = torch.stack([(torch.arange(length) + mod) % 7 < 4 for mod in mods])
ac_mask = torch.stack([(torch.arange(length) + mod) % 7 == 4 for mod in mods])
rw_mask = torch.stack([(torch.arange(length) + mod) % 7 == 5 for mod in mods])
rtg_mask = torch.stack([(torch.arange(length) + mod) % 7 == 6 for mod in mods])

test = torch.tensor(np.random.rand(3, 14, 5), dtype=torch.float)
# print(test)
print(test.shape, st_mask.shape)
# print(test[rtg_mask].reshape(3, -1, 5))
# print(test[rtg_mask].shape)

test[rtg_mask] = torch.zeros((6, 5), dtype=torch.float)

In [ ]:
print(test)

In [ ]:
out[st_mask] = torch.zeros(st_mask.shape)